In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_17667/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_17667/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_19_5,0.898243,0.709169,0.736521,0.848416,0.033780,0.172650,0.184572,0.060174,0.078013,0.183793,0.882797,0.185223,40.775790,89.523875,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1,model_1_19_3,0.898336,0.709164,0.735975,0.849244,0.033749,0.172653,0.184955,0.059845,0.078003,0.183709,0.882904,0.185138,40.777614,89.525700,"Hidden Size=[4], regularizer=0.02, learning_ra..."
2,model_1_19_0,0.898473,0.709147,0.735129,0.850513,0.033703,0.172663,0.185547,0.059342,0.077988,0.183585,0.883063,0.185013,40.780320,89.528406,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_1_18_7,0.899708,0.707225,0.720783,0.915747,0.033293,0.173804,0.195597,0.026273,0.078163,0.182464,0.884486,0.183884,40.804808,89.552894,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4,model_1_18_3,0.899671,0.707072,0.720703,0.916655,0.033306,0.173895,0.195653,0.025990,0.078169,0.182498,0.884443,0.183918,40.804065,89.552151,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,model_7_0_0,-0.247666,-0.122343,-0.290856,-0.238192,0.414180,0.666271,0.904271,0.448824,0.316294,0.643568,-1.514827,0.648576,131.762910,318.152650,"Hidden Size=[8, 4], regularizer=0.02, learning..."
1496,model_9_0_0,-0.036224,-0.128692,-0.146013,0.104642,0.343989,0.670040,0.802806,0.237334,0.295642,0.586505,1.522160,0.591070,772.134293,1876.135057,"Hidden Size=[24, 12], regularizer=0.02, learni..."
1497,model_5_0_0,-0.024530,-0.197048,-0.433733,0.121401,0.340107,0.710620,1.004360,0.310611,0.312766,0.583187,9.260274,0.587725,292.156992,707.949487,"Hidden Size=[36], regularizer=0.02, learning_r..."
1498,model_10_0_0,-0.085831,-0.214586,-0.528609,-0.425592,0.360456,0.721031,1.070822,0.498942,0.298311,0.600380,1.210952,0.605053,1588.040768,3861.995587,"Hidden Size=[36, 18], regularizer=0.02, learni..."
